<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [1]:
from sqlalchemy import create_engine
import sqlalchemy
import psycopg2

import pandas as pd

import requests 
from pprint import pprint 
from bs4 import BeautifulSoup 

import warnings
warnings.filterwarnings('ignore')

In [2]:
# вставьте сюда параметры подключения из юнита 1. Работа с базой данных из Python
DBNAME = 
USER = 
PASSWORD = 
HOST = 
PORT = 

In [3]:
connection = psycopg2.connect(
   dbname=DBNAME,
   user=USER,
   host=HOST,
   password=PASSWORD,
   port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [4]:
# текст запроса
query_3_1 = f'''
            SELECT 
               COUNT(id)  "Общее количество вакансий"
            FROM 
               public.vacancies
        '''


In [5]:
# результат запроса
hh_df = pd.read_sql_query(query_3_1, connection)
hh_df

,Общее количество вакансий
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [6]:
# текст запроса
query_3_2 = f'''
            SELECT 
               COUNT(id) "Общее количество работодателей"
            FROM 
               public.employers
        '''

In [7]:
# результат запроса
hh_df = pd.read_sql_query(query_3_2, connection)
hh_df

,Общее количество работодателей
0,23501


3. Посчитайте с помощью запроса количество регионов (таблица areas).

In [8]:
# текст запроса
query_3_3 = f'''
            SELECT 
               COUNT(id) "Общее количество регионов"
            FROM 
               public.areas
        '''

In [9]:
# результат запроса
hh_df = pd.read_sql_query(query_3_3, connection)
hh_df

,Общее количество регионов
0,1362


4. Посчитайте с помощью запроса количество сфер деятельности в базе (таблица industries).

In [10]:
# текст запроса
query_3_4 = f'''
            SELECT 
               COUNT(id) "Количество сфер деятельности"
            FROM 
               public.industries
        '''

In [11]:
# результат запроса
hh_df = pd.read_sql_query(query_3_4, connection)
hh_df

,Количество сфер деятельности
0,294


***

Выводы

1. В среднем, на одного работодателя приходится по 2 вакансии.В свою очередь размещение более одной вакансии может указывать на различные причины:
- большие объемы работы не могут быть выполнены одним работником;
- работодатель "перестраховывается" и производит набор сразу двух работников для выбора лучшего по результатам испытательного срока;
- работодатель "закрывает" потребности отдела, нанимая сразу двух работников, возможно с разным опытом работы и должностью, либо с разными навыками;
- набор производится в том числе в филиалы организации.
2. При этом следует учитывать следующий фактор: в базе www.HH.ru есть устаревшие, т.н. "мертвые" вакансии, что сразу уменьшает количество вакансий на одного работодателя.
3. Поэтому без изучения других условий, в частности, географии работодателей, можно сделать только осторожные выводы, которые нельзя использовать для анализа рынка труда.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [12]:
# текст запроса
query_4_1 = f'''
            SELECT 
                COUNT(v.id) "Количество вакансий",
                a.name "Регион"
            FROM 
                public.vacancies v
            JOIN public.areas a ON v.area_id=a.id 
            GROUP BY a.id 
            ORDER BY 1 DESC
            LIMIT 5
        '''

In [13]:
# результат запроса
hh_df = pd.read_sql_query(query_4_1, connection)
hh_df

,Количество вакансий,Регион
0,5333,Москва
1,2851,Санкт-Петербург
2,2112,Минск
3,2006,Новосибирск
4,1892,Алматы


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [14]:
# текст запроса
query_4_2 = f'''
            SELECT 
                COUNT(*) "Количество вакансий"
            FROM 
                public.vacancies v
            WHERE  
                salary_to IS NOT NULL 
                OR 
                salary_from IS NOT NULL
        '''

In [15]:
# результат запроса
hh_df = pd.read_sql_query(query_4_2, connection)
hh_df

,Количество вакансий
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [16]:
# текст запроса
query_4_3 = f'''
            SELECT 
                round(avg(v.salary_from)):: int "Минимальная средняя зарплата",
                round(avg(v.salary_to))::int "Максимальная средняя зарплата"
            FROM 
                public.vacancies v
            WHERE  
                salary_to IS NOT NULL 
                or 
                salary_from is NOT NULL     
        '''


In [17]:

# результат запроса
hh_df = pd.read_sql_query(query_4_3, connection)
hh_df

,Минимальная средняя зарплата,Максимальная средняя зарплата
0,71065,110537


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [18]:
# текст запроса
query_4_4 = f'''
            SELECT 
                concat (v.schedule, ' - ', v.employment) "Условия трудоустройства",
                COUNT(v.id) "Количество вакансий"
            FROM 
                public.vacancies v
            GROUP BY 1  
            ORDER BY 2 DESC
        '''

In [19]:
# результат запроса
hh_df = pd.read_sql_query(query_4_4, connection)
hh_df

,Условия трудоустройства,Количество вакансий
0,Полный день - Полная занятость,35367
1,Удаленная работа - Полная занятость,7802
2,Гибкий график - Полная занятость,1593
3,Удаленная работа - Частичная занятость,1312
4,Сменный график - Полная занятость,940
5,Полный день - Стажировка,569
6,Вахтовый метод - Полная занятость,367
7,Полный день - Частичная занятость,347
8,Гибкий график - Частичная занятость,312
9,Полный день - Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [20]:
# текст запроса
query_4_5 = f'''
            SELECT  
                v.experience "Опыт работы",
                COUNT(v.id) "Количество вакансий"
            FROM 
                public.vacancies v
            WHERE 
                v.experience IS NOT NULL
            GROUP BY 1 
            ORDER BY 2
        '''

In [21]:
# результат запроса
hh_df = pd.read_sql_query(query_4_5, connection)
hh_df

,Опыт работы,Количество вакансий
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

Выводы

1. Выборка ТОП-5 состоит из следующих крупных городов-работодателей: Москва, Санкт-Петербург, Минск, Новосибирск, Алматы. На Москву приходится 37,6% вакансий ТОПа, что ожидаемо.Вторая столица разместила 20,1% вакансий. Однако следует помнить, что на вакансию, размещенную, например, в Москве, может устроиться специалист из любого другого региона на удаленную работу. Поэтому сделать какой-либо вывод о занятости в регионах только по местонахождению работодателя нельзя: надо учитывать и удаленный формат работы, и процент реально переехавших из регионов специалистов в города-миллионники.
2. Не во всех вакансиях отражены финансовые условия работы в полном объеме, что указывает на готовность работодателя оговаривать точный размер зарплаты только после удачного прохождения соискателем собеседования.Одновременно с этим, внимание к таким вакансиям ниже.
3. Данные, полученные по размерам средних минимальной и максимальной заработной платы, без изучения соотношения с опытом, регионом, графиком работы/расписанием, невозможно интерпретировать детально. Вполне ожидаемо, что зарплата будет выше в столицах, при большем опыте работы. 
4. Соотношение количества вакансий с опытом работы достаточно интересно: больше всего вакансий приходится на молодых специалистов с опытом от одного до трех лет (53%). Это может указывать на предложение низкой зарплаты с дальнейшим ее ростом, либо предложение места помощника более зрелого специалиста, либо (что имеет место на практике) неперспективное место работы с дальнейшим расторжением договора и наймом нового молодого сотрудника.
Меньше всего вакансий размещено для соискателей с опытом более шести лет: либо такие сотрудники более требовательны к размеру зарплаты, либо большинство из них, как правило, имеют большой стаж работы на одном месте, т.е. значительно реже увольняются, что отражается на спросе на таких специалистов.
5. Данные по графику работу ожидаемы: на первом месте вакансии "Полный день - Полная занятость". Это соответствует менталитету работодателя еще советского времени:работодатели готовы платить больше за нахождение в офисе, нежели за выполняемую работу.При этом период короновируса стал для работодателей испытанием с получением нового опыта: потеря контроля над работником из-за вынужденного перевода его на удаленную работу  некоторым работодателям открыла возможность получения не меньших результатов от работников и экономии затрат на офисные условия. Так же некоторые работодатели вынуждены уступить тенденции в желании работника осуществлять свои трудовые обязанности удаленно при полной занятости (такие вакансии заняли второе место).

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [22]:
# текст запроса
query_5_1 = f'''
            (SELECT
                e.name "Наименование работодателя", 
                COUNT(*) "Количество вакансий"
            FROM vacancies v
            JOIN employers e ON v.employer_id = e.id
            GROUP BY e.name
            ORDER BY 2 DESC
            LIMIT 1)
            UNION ALL
            (SELECT 
                e.name "Наименование работодателя", 
                COUNT(*) "Количество вакансий"
            FROM vacancies v
            JOIN employers e ON v.employer_id = e.id
            GROUP BY e.name
            ORDER BY 2 DESC
            OFFSET 4 
            LIMIT 1)
        '''

In [23]:
# результат запроса
hh_df = pd.read_sql_query(query_5_1, connection)
hh_df

,Наименование работодателя,Количество вакансий
0,Яндекс,1933
1,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [24]:
# текст запроса
query_5_2 = f'''
            SELECT 
                a.name "Регион",
                COUNT(e.id) "Количество работодателей",
                COUNT(v.id) "Количество вакансий"
            FROM 
                public.areas a
            LEFT JOIN public.vacancies v ON v.area_id=a.id 
            LEFT JOIN public.employers e ON e.area=a.id
            WHERE 
                v.id IS NULL
            GROUP BY 1   
            ORDER BY 2 DESC 
            LIMIT 1
        '''

In [25]:
# результат запроса
hh_df = pd.read_sql_query(query_5_2, connection)
hh_df

,Регион,Количество работодателей,Количество вакансий
0,Россия,410,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [26]:
# текст запроса
query_5_3 = f'''
            SELECT 
                e.name "Наименование работодателя",
                COUNT(DISTINCT v.area_id) "Количество регионов"
            FROM 
                public.vacancies v
            JOIN public.employers e ON v.employer_id=e.id  
            GROUP BY 1  
            ORDER BY 2 DESC
        '''

In [27]:
# результат запроса
hh_df = pd.read_sql_query(query_5_3, connection)
hh_df

,Наименование работодателя,Количество регионов
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
14761,UniSol,1
14762,UNISTORY LLC,1
14763,UNIT6,1
14764,United Distribution,1


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [28]:
# текст запроса
query_5_4 = f'''
            SELECT 
                COUNT(e.id) "Количество работодателей"
            FROM 
                public.employers e
            LEFT JOIN public.employers_industries ei ON ei.employer_id=e.id  
            WHERE 
                ei.industry_id IS NULL
        '''

In [29]:
# результат запроса
hh_df = pd.read_sql_query(query_5_4, connection)
hh_df

,Количество работодателей
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [30]:
# текст запроса
query_5_5 = f'''
            SELECT  
                e.name "Наименование работодателя",
                COUNT(*) "Количество сфер деятельности"
            FROM 
                public.employers_industries ei
            JOIN public.employers e ON ei.employer_id=e.id  
            GROUP BY 1 
            HAVING COUNT(ei.industry_id) = 4  
            ORDER BY 1
            LIMIT 3 
            
            
        '''

In [31]:
# результат запроса
hh_df = pd.read_sql_query(query_5_5, connection)
hh_df

,Наименование работодателя,Количество сфер деятельности
0,101 Интернет,4
1,21vek.by,4
2,2ГИС,4


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [32]:
# текст запроса
query_5_6 = f'''
            SELECT  
                COUNT(e.id) "Количество работодателей"
            FROM 
                public.employers e
            JOIN public.employers_industries ei ON ei.employer_id=e.id  
            JOIN public.industries i ON ei.industry_id=i.id  
            WHERE i.name LIKE '%%Разработка программного обеспечения%%'
        '''

In [33]:
# результат запроса
hh_df = pd.read_sql_query(query_5_6, connection)
hh_df

,Количество работодателей
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим заданием посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [34]:
url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8' # Задаём адрес ресурса
response = requests.get(url)

hh_df = pd.DataFrame(pd.read_html(response.text)[1])
cities = tuple(hh_df['Город'].apply(lambda x: x.strip('[a]')))


In [35]:
# текст запроса 
query_5_7 = f'''
            (SELECT  
                a.name "Название города",
                COUNT(v.id) "Количество вакансий"
            FROM 
                public.areas a
            JOIN public.vacancies v ON v.area_id=a.id  
            JOIN public.employers e ON e.id=v.employer_id
            WHERE 
                a.name IN {cities} AND e.name='Яндекс' 
            GROUP BY a.id)
            UNION ALL  
            (SELECT 
                'Всего вакансий',
                COUNT(v.id)
            FROM 
                public.areas a
            JOIN public.vacancies v ON v.area_id=a.id
            JOIN public.employers e ON e.id=v.employer_id
            WHERE 
                a.name IN {cities} AND e.name='Яндекс')
           ORDER BY 2
        '''


In [36]:
# результат запроса
hh_df = pd.read_sql_query(query_5_7, connection)
hh_df

,Название города,Количество вакансий
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Самара,26
8,Уфа,26
9,Краснодар,30


***

Выводы

1. Вакансии в первой десятке принадлежат крупным компаниям. Обращает на себя внимание некая зависимость нахождения в рейтинге от степени участия компании /использования в своей деятельности продуктов IT.  На первом месте по количеству вакансий заслуженно находится Яндекс, компания с широким спектром услуг, в том числе в сфере IT.На втором месте располагается Ростелеком, который широко представлен в этой же сфере. А вот компании Тензор и Контур, представляющие услуги в более узкой сфере, несмотря на большую сеть филиалов, меньше нуждаются в новых сотрудниках.
2. Указание России в качестве своего региона создает затруднения в изучении вакансий. Хотя такой вариант все же возможен в случае размещения вакансии с удаленным форматом работы, хотя и не является корректным. 
3. Результаты запроса подтверждают версию о разнообразии услуг, оказываемых работодателем, а также его представленности в регионах, что свойственно крупным игрокам на рынке. Лидирует Яндекс с охватом 181 региона.
4. Почти каждый третий работодатель по неизвестной причине не указывает в вакансиях сферу деятельности. Если крупные компании известны, то мелкие игроки либо делающие первые шаги в бизнесе делают большую ошибку, не считая эту информацию важной. При размещении вакансии необходимо учитывать, что соискатели, желающие построить карьеру в конкретной профессии, часто ищут вакансии у работодателей, которые специализируются в этой сфере деятельности. Для некорьериста сфера деятельности работодателя не так важна, но такой специалист скорее всего будет иметь меньшее стремление совершенствоваться, что может отразиться в последствии на качестве работы.
5. Количество сфер деятельности напрямую зависит от количества услуг/товаров, которые компания может предложить потребителям/бизнесу.Чем меньше сфер деятельности, тем выше должна быть востребована услуга/товар этой сферы.Например, у 1С, оказывающей услуги автоматизации бизнес-процесса предприятия, программное обеспечение общепризнано и не имеет аналогов в РФ. Хорошим признаком является наличие нескольких сфер деятельности, взаимосвязанных и дополняющих друг друга. У 2ГИС 4 сферы деятельности, у Билайна -6, что соответствует информации, известной о разнообразии оказываемых ими услуг.При изучении работодателей интересно отметить одинаковое количество сфер деятельности у компаний, занимающихся фармацевтикой и медизделиями: у Abbott, Bayer, B.Braun по 5 сфер. При этом в Alcon, продающей контактные линзы, сфер тоже 5. Возможно, такое количество связано с требованиями лицензирования деятельности.
6. Среди всех работодателей, в качестве сферы деятельности Разработку Программного обеспечения указали 3553 работодателя, что составляет 15,12%. При этом работодатели, услугами которых мы пользуемся ежедневно (браузеры, социальные сети, платежные системы и прочее), но указавшие в качестве сферы деятельности "Интернет-компании", значительно меньше - 1675 (7,13%).
7. Распределение вакансий Яндекса по городам-миллионникам четко зависит от количества населения и статуса города: столицы занимают первые места.Из 485 вакансий 11,13% приходится на Москву, а 8,66%- на Санкт-Петербург.

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [37]:
# текст запроса
query_6_1 = f'''
            SELECT  
                COUNT(v.id) "Количество вакансий"
            FROM
                public.vacancies v
            WHERE 
                LOWER(v.name) LIKE '%%data%%' OR LOWER(v.name) LIKE '%%данн%%'
        '''

In [38]:
# результат запроса
hh_df = pd.read_sql_query(query_6_1, connection)
hh_df

,Количество вакансий
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [39]:
# текст запроса
query_6_2 = f'''
            SELECT 
                COUNT(*) "Количество вакансий"
            FROM 
                public.vacancies
            WHERE
                (LOWER(name) LIKE '%%data scientist%%' OR 
                LOWER(name) LIKE '%%data science%%' OR
                LOWER(name) LIKE '%%исследователь данных%%' OR
                LOWER(name) LIKE '%%machine learning%%' OR
                LOWER(name) LIKE '%%машинн%%обучен%%' OR
                (name NOT LIKE '%HTML%' AND name LIKE '%ML%'))
                AND (LOWER(name) LIKE '%%junior%%'  
                OR experience='Нет опыта'
                OR employment='Стажировка')
        '''
        

In [40]:
# результат запроса
hh_df = pd.read_sql_query(query_6_2, connection)
hh_df

,Количество вакансий
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [41]:
# текст запроса
query_6_3 = f'''
            SELECT 
                COUNT(*) "Количество вакансий"
            FROM 
                public.vacancies
            WHERE
                (LOWER(name) LIKE '%%data scientist%%' OR 
                LOWER(name) LIKE '%%data science%%' OR
                LOWER(name) LIKE '%%исследователь данных%%' OR
                LOWER(name) LIKE '%%machine learning%%' OR
                LOWER(name) LIKE '%%машинн%%обучен%%' OR
                (name NOT LIKE '%HTML%' AND name LIKE '%ML%'))
                AND (UPPER(key_skills) LIKE '%%SQL%%'
                OR LOWER(key_skills) LIKE '%%postgres%%')
        '''

In [42]:
# результат запроса
hh_df = pd.read_sql_query(query_6_3, connection)
hh_df

,Количество вакансий
0,201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [43]:
# текст запроса
query_6_4 = f'''
            SELECT 
                COUNT(*) "Количество вакансий"
            FROM 
                public.vacancies
            WHERE
                (LOWER(name) LIKE '%%data scientist%%' OR 
                LOWER(name) LIKE '%%data science%%' OR
                LOWER(name) LIKE '%%исследователь данных%%' OR
                LOWER(name) LIKE '%%machine learning%%' OR
                LOWER(name) LIKE '%%машинн%обучен%%' OR
                (name NOT LIKE '%%HTML%%' AND name LIKE '%%ML%%'))
                AND LOWER(key_skills) LIKE '%%python%%'
        '''

In [44]:
# результат запроса 
hh_df = pd.read_sql_query(query_6_4, connection)
hh_df

,Количество вакансий
0,351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [45]:
# текст запроса
query_6_5 = f'''
            SELECT 
                (ROUND(AVG(LENGTH(key_skills) - LENGTH(REPLACE(key_skills, CHR(9), '')) + 1),2)) "Количество ключевых навыков"
            FROM 
                public.vacancies
            WHERE
                (LOWER(name) LIKE '%%data scientist%%' OR 
                LOWER(name) LIKE '%%data science%%' OR
                LOWER(name) LIKE '%%исследователь данных%%' OR
                LOWER(name) likE '%%machine learning%%' OR
                LOWER(name) LIKE '%%машинн%обучен%%' OR
                (name NOT LIKE '%%HTML%%' AND name LIKE '%%ML%%'))
                AND key_skills IS NOT NULL
        '''

In [46]:
#результат запроса
hh_df = pd.read_sql_query(query_6_5, connection)
hh_df

,Количество ключевых навыков
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [47]:
# текст запроса  
query_6_6 = f'''
            SELECT 
                experience "Опыт работы",
                AVG((coalesce(salary_from, salary_to) + coalesce(salary_to, salary_from))/2)::int "Размер зарплаты"
            FROM 
                public.vacancies
            WHERE
                (LOWER(name) LIKE '%%data scientist%%' OR 
                LOWER(name) LIKE '%%data science%%' OR
                LOWER(name) LIKE '%%исследователь данных%%' OR
                LOWER(name) LIKE '%%machine learning%%' OR
                LOWER(name) LIKE '%%машинн%обучен%%' OR
                (name NOT LIKE '%%HTML%%' AND name LIKE '%%ML%%'))
                AND (salary_from IS NOT NULL OR
                salary_to IS NOT NULL)
            GROUP BY 1
        '''

In [48]:
# результат запроса 
hh_df = pd.read_sql_query(query_6_6, connection)
hh_df

,Опыт работы,Размер зарплаты
0,Нет опыта,74643
1,От 1 года до 3 лет,139675
2,От 3 до 6 лет,243115


***

Выводы

1. Вакансий, имеющих отношение к работе с данными (основываясь на прямом упоминании ключевых слов в самом названии вакансии), в базе насчитывается 1771. Делать выводы на этом основании не следует, т.к. необходимые навыки для работы с данными могут быть указаны в key_skills.
2. Для начинающего карьеру в сфере DS в базе представлена 51 вакансия.
3. Знание SQL или postgres в качестве ключевого навыка указано в 201 вакансии. Более популярен из навыков -Python: 351 вакансия содержит знание этого языка как необходимый навык.
4. В среднем для DS работодатели указывают в вакансиях 6,41 ключевых навыков.
5. Размер средней заработной платы находится в зависимости от опыта работы: 74643  рублей для начинающего DS без опыта до 243115 рублей с опытом до 6 лет.

Следует отдельно отметить следующее. В данном задании производилась фильтрация вакансий по ключевым словам в их названии. Однако в список попали "лишние" вакансии, например, "Разработчик C++", у которого в навыках указан язык QML. Добавив в запрос в имени отсутствие C++ количество вакансий стало равно 469 в отличие от правильного ответа в 480 вакансий (информация к размышлению, поэтому решение задач в данном проекте сделано в соответствии с правильными ответами).

# Общий вывод по проекту

1. Специалисты по работе с данными в целом востребованы на рынке труда, на что указывает уровень зарплаты даже для специалиста без опыта работы. Основная масса вакансий рассчитана на молодых специалистов с опытом 1-3 лет.
2. Основными ключевыми навыками (среди более 6 в среднем) являются Python и SQL/Postgres.
3. Большая часть вакансии размещена крупными работодателями с большой сетью филиалов, ассортиментом оказываемых услуг, сфер деятельности, уровня технического развития.
4. Наличие вакансий в разных рагионах дает возможность трудоустраиваться не выезжая за пределы своего региона (в т.ч. на условиях удаленного формата работы), что снижает процент переездов в столицы.

## Дополнительное исследование

Посмотрим ТОП-10 работодателей больше всего размещающих вакансии для DS

In [49]:
query_7_1 = f'''
            SELECT 
                e.name "Наименование работодателя",
                COUNT(ds.id) "Количество DS-вакансий"
            FROM 
                (SELECT *
                FROM vacancies 
            WHERE LOWER(name) LIKE '%%data scientist%%' OR
                LOWER(name) LIKE '%%data science%%' OR
                LOWER(name) LIKE '%%исследователь данных%%' OR
                (name NOT LIKE '%%HTML%%' AND name LIKE '%%ML%%') OR
                LOWER(name) LIKE '%%machine learning%%' OR
                LOWER(name) LIKE '%%машинн%обучен%%') ds
            JOIN employers e ON e.id = ds.employer_id
            GROUP BY 1
            ORDER BY 2 DESC
            LIMIT 10
        '''

In [50]:
hh_df = pd.read_sql_query(query_7_1, connection)
hh_df

,Наименование работодателя,Количество DS-вакансий
0,СБЕР,37
1,Bell Integrator,25
2,Банк ВТБ (ПАО),18
3,VK,15
4,Positive Technologies,11
5,Яндекс,9
6,EvenBet Gaming,9
7,МегаФон,8
8,Andersen,7
9,Ozon,6


Вывод
1. Несмотря на то, что Яндекс занимает первую строчку по количеству вакансий в целом, лидирует в количеству регионов представления вакансий, при запросе о количестве вакансий в сфере DS он находится только на 6 месте. При этом в данном анализе лидирует СБЕР, который позже Яндекса принял решение модернизироваться и расширять цифровой формат услуг и ведение бизнеса, и, видимо, экстренно "нагоняет" упущенное.
2. На 10 месте находится популярный маркетплейс Озон, однако вакансий DS не очень много, что,видимо, связано со спецификой такого бизнеса, который не требует большого штата сотрудников в целом и услуг DS в режиме nonstop.

Работа с данными требует знаний английского языка. Сколько работодателей указали такой навык для специалистов в сфере DS?

In [51]:
query_7_2 = f'''
            (SELECT 
                'Количество вакансий с английским языком'"Условие выбора" ,
                COUNT(*) "Количество"
            FROM 
                public.vacancies
            WHERE
                (LOWER(name) LIKE '%%data scientist%%' OR 
                LOWER(name) LIKE '%%data science%%' OR
                LOWER(name) LIKE '%%исследователь данных%%' OR
                LOWER(name) LIKE '%machine learning%%' OR
                LOWER(name) LIKE '%%машинн%обучен%%' OR
                (name NOT LIKE '%%HTML%%' AND name LIKE '%%ML%%'))
                AND (key_skills LIKE '%Английский%' OR key_skills LIKE '%%English%%'))
              
            UNION ALL          
              
              
             (SELECT 
              'Всего вакансий' "Условие выбора",
                COUNT(*) 
                FROM 
                public.vacancies
            WHERE
                (LOWER(name) LIKE '%%data scientist%%' OR 
                LOWER(name) LIKE '%%data science%%' OR
                LOWER(name) LIKE '%%исследователь данных%%' OR
                LOWER(name) LIKE '%%machine learning%%' OR
                LOWER(name) LIKE '%%машинн%обучен%%' OR
                (name NOT LIKE '%%HTML%%' AND name LIKE '%%ML%%')))
            ORDER BY 2
        '''
    

In [52]:
hh_df = pd.read_sql_query(query_7_2, connection)
hh_df

,Условие выбора,Количество
0,Количество вакансий с английским языком,50
1,Всего вакансий,480


Вывод
1. Знание английского языка, как навык, указан только в 50 вакансиях из 480, что составляет 10,4%. 
2. При этом отсутствие такого навыка не указывает на его ненужность, возможно такой навык подразумевается по умолчанию.

In [ ]:
#закрываем соединение после окончания работы
connection.close()